In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
print(tf.__version__)
tf.enable_eager_execution()

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from gensim.models import word2vec
import gensim
import pandas as pd
import nltk
import unicodedata

import unicodedata
import re
import numpy as np
import os
import sys
import time
import logging
import sqlite3
import multiprocessing
from itertools import cycle
import warnings

warnings.filterwarnings('ignore')

/home/cbrom/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.12.0


In [2]:
# sql connection
sql_con = sqlite3.connect('/home/cbrom/workspace/2015-02.db')

In [3]:
start = time.time()
sql_data = pd.read_sql("SELECT parent, comment from parent_reply", sql_con)
print('Total time to read: ' + str((time.time() - start)) + ' secs')

Total time to read: 13.970335721969604 secs


In [4]:
print('Total rows: ' + str(len(sql_data)))

Total rows: 4506428


In [13]:
def clean_up_text(pd_pairs, col_name):
    start = time.time()
    
    cleaned_col = 'cleaned_' + col_name
    
    # normalize tabs, replace new lines
    pd_pairs[cleaned_col] = pd_pairs[col_name].apply(
    lambda col: str(col).replace('\t', ' ').replace('\n', ''))
    
    # remove all characters except A-Z and a dot.
    pd_pairs[cleaned_col] = pd_pairs[cleaned_col].apply(
    lambda cl_col: re.sub("[^a-zA-Z\.]", " ", cl_col))
    
    # shrink spaces
    pd_pairs[cleaned_col] = pd_pairs[cleaned_col].apply(
    lambda cl_col: re.sub("[ .]+", " ", cl_col))
    
    # strip
    pd_pairs[cleaned_col] = pd_pairs[cleaned_col].apply(
    lambda cl_col: cl_col.strip())
    
    # to lower case
    pd_pairs[cleaned_col] = pd_pairs[cleaned_col].apply(
    lambda cl_col: cl_col.lower())
    
    # change to unicode
    pd_pairs[cleaned_col] = pd_pairs[cleaned_col].apply(
    lambda cl_col: ''.join(c for c in unicodedata.normalize('NFD', cl_col)
                          if unicodedata.category(c) != 'Mn'))
    
    # create space between word and panctuation
    pd_pairs[cleaned_col] = pd_pairs[cleaned_col].apply(
    lambda cl_col: re.sub(r"([?.!,])", r" \1 ", cl_col))
    
    # add a start and an end token to sentence
    pd_pairs[cleaned_col] = pd_pairs[cleaned_col].apply(
    lambda cl_col: '<start> ' + cl_col + ' <end>')
    
    return

In [14]:
clean_up_text(sql_data, 'parent')

In [15]:
clean_up_text(sql_data, 'comment')

In [16]:
sql_data.head()

,parent,comment,cleaned_parent,cleaned_comment
0,&gt; CDBOOT: Cannot boot from CD - Code: 5 new...,Ok will do!,<start> gt cdboot cannot boot from cd code new...,<start> ok will do <end>
1,Sounds like someone wants some pants jerky,I'm an avid fan of pants meat.,<start> sounds like someone wants some pants j...,<start> i m an avid fan of pants meat <end>
2,"That's definitely a valid point, but I would a...",But where do we draw the line on frequency? An...,<start> that s definitely a valid point but i ...,<start> but where do we draw the line on frequ...
3,I feel like Jotaro would like Beat-em-ups more...,In part 6 he starts getting into Octodad,<start> i feel like jotaro would like beat em ...,<start> in part he starts getting into octodad...
4,I think we've just discovered the fandom's sbu...,Land of Pictures and Text,<start> i think we ve just discovered the fand...,<start> land of pictures and text <end>


In [17]:
cleaned_data = sql_data[['cleaned_parent', 'cleaned_comment']]

In [18]:
cleaned_data.to_csv('cleaned_data.csv', index=False)

-----------------------------